In [19]:
%matplotlib inline
from deco.imports import *
import json
from pprint import pprint
import sklearn.ensemble as st
from datetime import datetime
import random
from math import sin, cos, sqrt, atan2, radians
import networkx as nx

In [2]:
PATH = '/home/rotem/Documents/code/optimization-course/'

In [5]:
d = pd.read_csv(PATH + 'nyc-weather.csv')
d[d.NAME == 'NY CITY CENTRAL PARK, NY US'][['TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'AWND', 'WDF2', 'WDF5', 'WSF2', 'WSF5']]

,TMAX,TMIN,PRCP,SNOW,SNWD,AWND,WDF2,WDF5,WSF2,WSF5
19964,40.0,26.0,0.0,0.0,0.0,6.93,310.0,300.0,15.0,25.9
19965,33.0,22.0,0.0,0.0,0.0,5.82,310.0,340.0,15.0,21.9
19966,32.0,24.0,0.0,0.0,0.0,4.47,260.0,260.0,13.0,19.9
19967,37.0,30.0,0.0,0.0,0.0,8.05,290.0,250.0,17.9,28.0
19968,42.0,32.0,0.0,0.0,0.0,6.71,310.0,310.0,17.0,25.9
...,...,...,...,...,...,...,...,...,...,...
20325,55.0,36.0,0.0,0.0,0.0,7.16,250.0,240.0,16.1,27.1
20326,48.0,41.0,1.2,0.0,0.0,6.04,60.0,320.0,16.1,25.9
20327,45.0,23.0,0.0,0.0,0.0,6.26,300.0,300.0,15.0,25.1
20328,32.0,21.0,0.0,0.0,0.0,5.82,290.0,280.0,17.0,28.0


In [33]:
def coordinates_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(float(lat1))
    lon1 = radians(float(lon1))
    lat2 = radians(float(lat2))
    lon2 = radians(float(lon2))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [71]:
df = pd.read_csv(PATH + '../traffic-ds/data/mta_1706.csv', error_bad_lines=False, warn_bad_lines=False, nrows=1000000)

In [77]:
def filter_close_by(df, lat='OriginLat', long='OriginLong', count='RecordedAtTime', max_distance=0.05):
    org = df[[lat, long, count]].groupby([lat, long], as_index=False).count()
    res = []
    for row in org.values:
        for other in org.values:
            distance = coordinates_distance(row[0], row[1], other[0], other[1])
            if distance <= max_distance and distance > 0:
                res.append([row[0], row[1], other[0], other[1], distance, row[2], other[2], 1])
    distances = pd.DataFrame(res, columns=['lat1', 'long1', 'lat2', 'long2', 'distance', 'w1', 'w2', 'count'])
    return distances

In [82]:
def sort_by_weight(distances):
    distances_agg = distances.groupby(['lat1', 'long1'], as_index=False)\
                    .agg({'w2': 'sum', 'distance': 'mean', 'w1': 'max', 'count': 'sum'})
    distances_agg['weight'] = distances_agg.w1 + distances_agg.w2
    distances_agg = distances_agg.sort_values(['weight'], ascending=False)
    return distances_agg

In [83]:
distances = filter_close_by(df)
sort_by_weight(distances)

,lat1,long1,w2,distance,w1,count,weight
74,40.643429,-74.073654,16711.0,0.027461,2299.0,8,19010.0
75,40.643475,-74.073555,17029.0,0.026574,1981.0,8,19010.0
76,40.643490,-74.073441,16412.0,0.024872,1293.0,7,17705.0
114,40.709541,-73.959335,15002.0,0.025091,203.0,5,15205.0
113,40.709518,-73.959496,12377.0,0.018058,2828.0,5,15205.0
...,...,...,...,...,...,...,...
52,40.629044,-74.116272,15.0,0.026175,4.0,3,19.0
49,40.626667,-74.015038,4.0,0.034803,4.0,1,8.0
48,40.626450,-74.015335,4.0,0.034803,4.0,1,8.0
54,40.630280,-74.088814,5.0,0.011475,1.0,1,6.0


## Analyze by Connected components - not really doing anything

In [23]:
g = nx.Graph()
edges = [(f'{row[0]}_{row[1]}', f'{row[2]}_{row[3]}', row[4]) for row in df.values]
g.add_weighted_edges_from(edges)

In [36]:
import itertools
for i, comp in enumerate(nx.algorithms.connected_components(g)):
    print(f'******* Component {i} **********')
    print(f'Component size: {len(comp)}')
    
    distances = [coordinates_distance(*(x.split('_', 1) + y.split('_', 1))) 
                 for x, y in list(itertools.product(comp, comp))]
    depth = max(distances)
    print(f'Component depth: {depth}km')
    
## the largeset component's data is really far away from each other, hence it is not usable

******* Component 0 **********
Component size: 10
Component depth: 0.09176355391575176km
******* Component 1 **********
Component size: 4
Component depth: 0.029003338518161913km
******* Component 2 **********
Component size: 3
Component depth: 0.04244682175196422km
******* Component 3 **********
Component size: 4
Component depth: 0.09065483659090774km
******* Component 4 **********
Component size: 2
Component depth: 0.04846937214987653km
******* Component 5 **********
Component size: 3
Component depth: 0.0650748814414627km
******* Component 6 **********
Component size: 2
Component depth: 0.038912677669168484km
******* Component 7 **********
Component size: 2
Component depth: 0.017460071780791512km
******* Component 8 **********
Component size: 3
Component depth: 0.048821449082078035km
******* Component 9 **********
Component size: 6
Component depth: 0.0401894279075165km
******* Component 10 **********
Component size: 3
Component depth: 0.0418487378196112km
******* Component 11 ********